# base_model

> In this module we have the base model, learner and other things we need to train encoder
 

In [ ]:
#| default_exp base_model

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export

import self_supervised
import torch
from fastai.vision.all import *
from self_supervised.augmentations import *
from self_supervised.layers import *
import kornia.augmentation as korniatfm
import torchvision.transforms as tvtfm
from base_rbt.utils import *

In [ ]:
default_device??

Signature: default_device(use=-1)
Source:   
def default_device(use=-1):
    "Return or set default device; `use_cuda`: -1 - CUDA/mps if available; True - error if not available; False - CPU"
    if use == -1: use = defaults.use_cuda
    else: defaults.use_cuda=use
    if use is None:
        if torch.cuda.is_available(): use = True
    if use:
        if torch.cuda.is_available(): return torch.device(torch.cuda.current_device())
        if _has_mps(): return torch.device('mps')
    return torch.device('cpu')
File:      ~/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/fastai/torch_core.py
Type:      function

Here we have the base functions and classes to train a basic BT-style model. Note that this (mostly) all comes directly from here: `https://github.com/KeremTurgutlu/self_supervised/blob/main/nbs/14%20-%20barlow_twins.ipynb`
but we needed to extend some of the functionality for our purposes. 

Some of the base classes and functions needed for image augmentation pipeline:

In [ ]:
#| export

#My edited version of RandTransform
class RandomGaussianBlur(RandTransform):
    "Randomly apply gaussian blur with probability `p` with a value of s"
    order = 11
    def __init__(self, 
                 p=1.0, #debugging (bug in libraries implementation)
                 prob=0.5,#the real probability
                 s=(8,32), #kernel
                 sig=None, #sig_val is either manually input OR
                 blur_r=(0.1,2),#is randomly chosen from uniform with these bounds
                 same_on_batch=False, 
                 **kwargs): 
        
        store_attr()
        super().__init__(p=p, **kwargs)

    def encodes(self, x:TensorImage):
        
        if isinstance(self.s, int):   s = (self.s,self.s)
        elif isinstance(self.s, tuple) or isinstance(self.s, list): s=self.s
     
        #Default for ImageNet from BYOL / BT papers
        if self.sig is None:
            sig_val = np.random.uniform(self.blur_r[0],self.blur_r[1])
        
        else:
            sig_val = self.sig
            

        tfm = korniatfm.RandomGaussianBlur(kernel_size=s,sigma=(sig_val,sig_val),same_on_batch=self.same_on_batch,p=self.prob)
        return tfm(x)

#Delete later: leaving for backward compatibility for now
# class RandomGaussianBlur(RandTransform):
#     "Randomly apply gaussian blur with probability `p` with a value of s"
#     order = 11
#     def __init__(self, p=0.5, s=(8,32), same_on_batch=False, **kwargs): 
#         store_attr()
#         super().__init__(p=p, **kwargs)
        
#     def encodes(self, x:TensorImage):
#         if isinstance(self.s, tuple): s = np.random.randint(*self.s)
#         if isinstance(self.s, list):  s = np.random.randint(*self.s)
#         if isinstance(self.s, int):   s = self.s
#         s2 = int(s/4)*2+1
#         tfm = korniatfm.RandomGaussianBlur((s2,s2),(s,s),same_on_batch=self.same_on_batch,p=1.) #p=1. is a bug
#                                             #kernel #sigma
        
#         return tfm(x)
    
def get_BT_batch_augs(size,
                    flip=True,crop=True,noise=True,rotate=True,jitter=True,bw=True,blur=True,solar=True, #Whether to use  given aug or not
                    resize_scale=(0.08, 1.0),resize_ratio=(3/4, 4/3),noise_std=0.025, rotate_deg=30,jitter_s=.6,blur_s=(4,32),blur_r=(0.1,2),blur_sig=None,sol_t=0.05,sol_a=0.05, #hps of diff augs
                    flip_p=0.5, rotate_p=0.3,noise_p=0.2, jitter_p=0.3, bw_p=0.3, blur_p=0.3,sol_p=0.1, #prob of performing aug
                    same_on_batch=False,stats=imagenet_stats,cuda=default_device().type == 'cuda',xtra_tfms=[]):
    "Input batch augmentations implemented in tv+kornia+fastai"
    
    tfms = []

    korniatfm.RandomHorizontalFlip.order = RandomResizedCrop.order-1
    
    if crop: tfms += [tvtfm.RandomResizedCrop((size, size), scale=resize_scale, ratio=resize_ratio)]
    
    #Unfortunately for some reason this doesn't work, so we can't apply "same_on_batch=False"
    #tfms += [korniatfm.RandomResizedCrop((size, size), scale=resize_scale, ratio=resize_ratio, same_on_batch=same_on_batch)]
    
    if flip: tfms += [korniatfm.RandomHorizontalFlip(p=flip_p,same_on_batch=same_on_batch)]

    if rotate: tfms += [Rotate(max_deg=rotate_deg, p=rotate_p, batch=same_on_batch)]

                                             #brightness,contrast,saturation,hue
    if jitter: tfms += [korniatfm.ColorJitter(0.4*jitter_s, 0.4*jitter_s, 0.2*jitter_s, 0.1*jitter_s, p=jitter_p, same_on_batch=same_on_batch)]
    
    if bw:     tfms += [korniatfm.RandomGrayscale(p=bw_p, same_on_batch=same_on_batch)]
        
    #sig will usually be None
    if blur:   tfms += [RandomGaussianBlur(prob=blur_p, s=blur_s,sig=blur_sig,blur_r=blur_r, same_on_batch=same_on_batch)]

    korniatfm.RandomSolarize.order = RandomGaussianBlur.order + 1 #we want to apply solarization after RandomGaussianBlur
    
    if solar: tfms += [korniatfm.RandomSolarize(p=sol_p,thresholds=sol_t, additions=sol_a,same_on_batch=same_on_batch)]

    
    if noise: tfms+=[korniatfm.RandomGaussianNoise(mean=0.0, std=noise_std, same_on_batch=False, p=noise_p)]
    
    if stats is not None: tfms += [Normalize.from_stats(*stats, cuda=cuda)]

    tfms += xtra_tfms

    pipe = Pipeline(tfms, split_idx = 0)
    return pipe

@delegates(get_BT_batch_augs)
def get_multi_aug_pipelines(size, **kwargs): return get_BT_batch_augs(size, **kwargs)

@delegates(get_multi_aug_pipelines)
def get_barlow_twins_aug_pipelines(size,**kwargs): return get_multi_aug_pipelines(size=size,**kwargs)


Base functions / classes we need to train a BT / RBT model.

In [ ]:
#| export
#Base functions / classes we need to train a BT / RBT model.

#TODO: We can make these more abstract so can incrementally modify to build `bt/rbt` and also `new idea.` But for 
#sake of readability, might be easier to just modify the defintions elsewhere. Come back to this later...
class BarlowTwinsModel(Module):
    """An encoder followed by a projector
    """
    def __init__(self,encoder,projector):
        self.encoder = encoder
        self.projector = projector
        
    def forward(self,x): 
        
        return self.projector(self.encoder(x))

def create_barlow_twins_model(encoder, hidden_size=256, projection_size=128, bn=True, nlayers=3):
    "Create Barlow Twins model"
    n_in  = in_channels(encoder)
    with torch.no_grad(): representation = encoder(torch.randn((2,n_in,128,128)))
    projector = create_mlp_module(representation.size(1), hidden_size, projection_size, bn=bn, nlayers=nlayers) 
    apply_init(projector)
    return BarlowTwinsModel(encoder, projector)

class BarlowTwins(Callback):
    order,run_valid = 9,True
    def __init__(self, aug_pipelines,n_in, model_type='barlow_twins',lmb=5e-3, print_augs=False):
        assert_aug_pipelines(aug_pipelines)
        self.aug1, self.aug2 = aug_pipelines
        if print_augs: print(self.aug1), print(self.aug2)
        store_attr('lmb')
        self.n_in=n_in
        self.model_type = model_type
        self.index=-1 #Gets updated after each batch
        self.seed = np.random.randint(0,10000) #gets updated after each batch
        self.acc_dict = {}
        
    def before_fit(self): 
        self.learn.loss_func = self.lf
        nf = self.learn.model.projector[-1].out_features
        self.I = torch.eye(nf).to(self.dls.device)


    def before_epoch(self):
        self.index=-1  
  
    def before_batch(self):
        
        #TODO: Make this nicer (possibly can load in data as TensorImage(BW) or something?)
        #This is a bit of a hack. Can make this more elegant later. But in new version of FastAI
        #seems we need to compute TensorImage(BW) here, and depends on whether color or not, i.e. n_in.
        if self.n_in == 1:

            xi,xj = self.aug1(TensorImageBW(self.x)), self.aug2(TensorImageBW(self.x))
            
            #print(xi.shape)
                                    
        elif self.n_in == 3:
            
            xi,xj = self.aug1(TensorImage(self.x)), self.aug2(TensorImage(self.x))

        self.learn.xb = (torch.cat([xi, xj]),)

        self.index=self.index+1
        
    @torch.no_grad()
    def show(self, n=1): 
        bs = self.learn.x.size(0)//2
        x1,x2  = self.learn.x[:bs], self.learn.x[bs:]
        idxs = np.random.choice(range(bs),n,False)
        x1 = self.aug1.decode(x1[idxs].to('cpu').clone()).clamp(0,1)
        x2 = self.aug2.decode(x2[idxs].to('cpu').clone()).clamp(0,1)
        images = []
        for i in range(n): images += [x1[i],x2[i]]
        return show_batch(x1[0], None, images, max_n=len(images), nrows=n)

In [ ]:
#| export
#Here we give the model API for `new idea` or `RAT` -> i.e. two projector networks

#TODO: We can make these more abstract so can incrementally modify to build `bt/rbt` and also `new idea.` But for 
#sake of readability, might be easier to just modify the defintions elsewhere. Come back to this later...
class P2BarlowTwinsModel(Module):
    """An encoder followed by a projector
    """
    def __init__(self,encoder,projector,projector2):
        self.encoder = encoder
        self.projector = projector
        self.projector2 = projector2
        
    def forward(self,x): 
        tem = self.encoder(x)
        return self.projector(tem),self.projector2(tem)

def create_p2barlow_twins_model(encoder, hidden_size=256, projection_size=128, bn=True, nlayers=3):
    "Create Barlow Twins model"
    n_in  = in_channels(encoder)
    with torch.no_grad(): representation = encoder(torch.randn((2,n_in,128,128)))
    
    projector = create_mlp_module(representation.size(1), hidden_size, projection_size, bn=bn, nlayers=nlayers) 
    apply_init(projector)
    
    projector2 = create_mlp_module(representation.size(1), hidden_size, projection_size, bn=bn, nlayers=nlayers) 
    apply_init(projector2)
    
    
    return P2BarlowTwinsModel(encoder, projector,projector2)


In [ ]:
#| export
#We want access to both representation and projection

#TODO: We can make these more abstract so can incrementally modify to build `bt/rbt` and also `new idea.` But for 
#sake of readability, might be easier to just modify the defintions elsewhere. Come back to this later...
class P3BarlowTwinsModel(Module):
    """An encoder followed by a projector
    """
    def __init__(self,encoder,projector):
        self.encoder = encoder
        self.projector = projector
        
    def forward(self,x): 
        tem = self.encoder(x)
        return tem,self.projector(tem)
    
    def __str__(self):
        return 'forward returns tuple of (encoder(x),projector(encoder(x)))'

def create_p3barlow_twins_model(encoder, hidden_size=256, projection_size=128, bn=True, nlayers=3):
    "Create Barlow Twins model"
    n_in  = in_channels(encoder)
    with torch.no_grad(): representation = encoder(torch.randn((2,n_in,128,128)))
    
    projector = create_mlp_module(representation.size(1), hidden_size, projection_size, bn=bn, nlayers=nlayers) 
    apply_init(projector)
    
 
    return P3BarlowTwinsModel(encoder, projector)



In [ ]:
#| export

class P4BarlowTwinsModel(Module):
    """An encoder followed by a projector
    """
    def __init__(self,encoder,encoder2,projector,projector2):
        self.encoder = encoder
        self.encoder2 = encoder2
        self.projector = projector
        self.projector2 = projector2
        
    def forward(self,x):
        
        #If we have multiple GPUs can take advantage of it here...
        tem,tem2 = self.encoder(x),self.encoder2(x)
        return self.projector(tem),self.projector2(tem2)

def create_p4barlow_twins_model(encoder,encoder2, hidden_size=256, projection_size=128, bn=True, nlayers=3):
    "Create Barlow Twins model"
    n_in  = in_channels(encoder)
    with torch.no_grad(): representation = encoder(torch.randn((2,n_in,128,128)))
    
    projector = create_mlp_module(representation.size(1), hidden_size, projection_size, bn=bn, nlayers=nlayers) 
    apply_init(projector)
    
    projector2 = create_mlp_module(representation.size(1), hidden_size, projection_size, bn=bn, nlayers=nlayers) 
    apply_init(projector2)
    
    
    return P4BarlowTwinsModel(encoder,encoder2, projector,projector2)


Here we write down standard definition of `lf` for `RAT` method: 

BarlowTwins needs an `lf` method to work properly. Here we provide the `lf` of standard barlow twins. Later we can
patch in a new defintion of `lf` that involves random functions, inner maximization etc. The tools needed to do this are provised in `base_lf`

In [ ]:
#| export
def lf_bt(pred,I,lmb):
    bs,nf = pred.size(0)//2,pred.size(1)
    
    z1, z2 = pred[:bs],pred[bs:] #so z1 is bs*projection_size, likewise for z2

    z1norm = (z1 - z1.mean(0)) / z1.std(0, unbiased=False)
    z2norm = (z2 - z2.mean(0)) / z2.std(0, unbiased=False)

    C = (z1norm.T @ z2norm) / bs 
    cdiff = (C - I)**2
    loss = (cdiff*I + cdiff*(1-I)*lmb).sum() 
    return loss

In [ ]:
#| export
def lf_rbt_sparse(pred,I,lmb,sparsity_level,
               ):

    pred_enc = pred[0]
    pred = pred[1]

    bs,nf = pred.size(0)//2,pred.size(1)

    #All standard, from BT
    z1, z2 = pred[:bs],pred[bs:] #so z1 is bs*projection_size, likewise for z2
    z1norm = (z1 - z1.mean(0)) / z1.std(0, unbiased=False)
    z2norm = (z2 - z2.mean(0)) / z2.std(0, unbiased=False)

    z1_enc, z2_enc = pred_enc[:bs],pred_enc[bs:]

    sparsity = lmb*0.5*(torch.abs(z1_enc).sum() + torch.abs(z2_enc).sum())

    C = (z1norm.T @ z2norm) / bs
    cdiff = (C - I)**2

    rr = cdiff*(1-I)*lmb #redundancy reduction term (scaled by lmb)

    loss = (cdiff*I + rr).sum() #standard bt loss

    loss = loss + sparsity_level*sparsity


    torch.cuda.empty_cache()
    return loss

In [ ]:
#| export

@patch
def lf(self:BarlowTwins, pred,*yb,sparsity_level=0.25):
    "Assumes model created according to type p3"

    if self.model_type=='barlow_twins':
         pred_enc = pred[0]
         pred = pred[1]
         return lf_bt(pred, self.I,self.lmb)

    elif self.model_type=='sparse_barlow_twins':
        return lf_rbt_sparse(pred, self.I,self.lmb,sparsity_level=sparsity_level)

    else: raise(Exception)

In [ ]:
#| export
def my_splitter_bt(m):
    return L(sequential(*m.encoder),m.projector).map(params)

Here we show how to use the above functions in an end to end fashion. First we get some data and plonk it into a dls, Then create an encoder, an augmentation pipeline, a learner, then fit
the learner. This is the complete process of training BT. 

In [ ]:
#| export
def show_bt_batch(dls,n_in,aug,n=2,print_augs=True):
    "Given a linear learner, show a batch"
        
    learn = Learner(dls,model=None, cbs=[BarlowTwins(aug,n_in=n_in,print_augs=print_augs)])
    b = dls.one_batch()
    learn._split(b)
    learn('before_batch')
    axes = learn.barlow_twins.show(n=n)

In [ ]:
#| export

class TrainBT:
    "Train model using BT."

    def __init__(self,
                 model,#An encoder followed by a projector
                 dls,
                 bt_aug_pipelines,
                 lmb,
                 n_in,
                 model_type,
                 wd,
                 device,
                 ):

        store_attr()
        self.learn = self.setup_learn()

    
    def setup_learn(self):
        """
        Sets up the learner with the model, callbacks, and metrics.

        Returns:
        - learn: The Learner object.
        """
      
        self.model.to(self.device)
  
        learn=Learner(self.dls,self.model,splitter=my_splitter_bt,wd=self.wd, cbs=[BarlowTwins(self.bt_aug_pipelines,n_in=self.n_in,lmb=self.lmb,print_augs=False,model_type=self.model_type)])

        return learn
    
    def bt_transfer_learning(self,freeze_epochs:int=1,epochs:int=1):
        """If the encoder is alreaady pretrained, we can do transfer learning.
            Freeze encoder, train projector for a few epochs, then unfreeze and train all. 
        """

        self.learn.freeze()
        test_grad_off(self.learn.encoder)
        self.learn.fit(freeze_epochs)
        self.learn.unfreeze()
        test_grad_on(self.learn.model)
        lrs = self.learn.lr_find()
        self.learn.fit_one_cycle(epochs, lrs.valley)

        return self.learn.model

    def bt_learning(self,epochs:int=1):
        """If the encoder is not pretrained, we can do normal training.
        """
        
        lrs = self.learn.lr_find()
        self.learn.fit_one_cycle(epochs, lrs.valley)

        return self.learn.model

def train_bt(model,#An encoder followed by a projector
            dls,
            bt_aug_pipelines,
            lmb,
            n_in,
            model_type,
            wd,
            epochs,
            freeze_epochs,
            weight_type,
            device,
            ):
    
    bt_trainer = TrainBT(model=model,dls=dls,bt_aug_pipelines=bt_aug_pipelines,
                        lmb=lmb,n_in=n_in,model_type=model_type,wd=wd,device=device
                        )

    if weight_type!='random':
        model=bt_trainer.bt_transfer_learning(freeze_epochs=freeze_epochs,epochs=epochs)
    else:
        model=bt_trainer.bt_learning(epochs=epochs)

    return model
    


Augmentations for CIFAR10

In [ ]:
#| export


def get_bt_cifar10_aug_pipelines(size):
    aug_pipelines_1 = get_barlow_twins_aug_pipelines(size=size,
                                                    bw=True, rotate=True,noise=True, jitter=True, blur=True,solar=True,
                                                    resize_scale=(0.4, 1.0),rotate_deg=45,noise_std=0.0125, jitter_s=1.0, blur_s=math.ceil(size/10)+1,
                                                    bw_p=0.2, flip_p=0.5,rotate_p=0.25,noise_p=0.5, jitter_p=0.5, blur_p=0.5,sol_p=0.0,
                                                    stats=cifar_stats,same_on_batch=False, xtra_tfms=[]
                                                    )

    aug_pipelines_2 = get_barlow_twins_aug_pipelines(size=size,
                                                    bw=True, rotate=True,noise=True, jitter=True, blur=True,solar=True,
                                                    resize_scale=(0.4, 1.0),rotate_deg=45,noise_std=0.0125, jitter_s=1.0, blur_s=math.ceil(size/10)+1,sol_t=0.01,sol_a=0.01,
                                                    bw_p=0.2, flip_p=0.5,rotate_p=0.25,noise_p=0.5, jitter_p=0.5, blur_p=0.1,sol_p=0.2,
                                                    stats=cifar_stats,same_on_batch=False, xtra_tfms=[]
                                                    )

    bt_cifar10_aug_pipelines = [aug_pipelines_1,aug_pipelines_2]

    return bt_cifar10_aug_pipelines

#Add other augmentations here e.g. BYOL augs

bt_aug_func_dict = {'bt_cifar10_aug_pipelines':get_bt_cifar10_aug_pipelines}


def get_bt_aug_pipelines(bt_augs,size):

    return bt_aug_func_dict[bt_augs](size)

    


End to end example

In [ ]:

# from google.colab import drive
# drive.mount('/content/drive')



# #Choose the config file: this is all we change to run different experiments
config_path = '../configs/cifar10/config1.yaml'
config = load_config(config_path)

#cuda or cpu
device = default_device()

###hps from config

#still need to figure out how to set lmb properly
lmb = 1/config.ps
#config.lmb = lmb

#Based on the config, construct model
encoder = resnet_arch_to_encoder(arch=config.arch,weight_type=config.weight_type)
model = create_p3barlow_twins_model(encoder, hidden_size=config.hs, projection_size=config.ps)

#Based on config, construct data
dls = get_ssl_dls(dataset=config.dataset,bs=config.bs, device=device)

#Based on config, construct augmentations
bt_aug_pipelines = get_bt_aug_pipelines(bt_augs=config.bt_augs, size=config.size)


# ###Setup saving?

# #Make directory based on config file? (this is on google drive)

@patch
def after_epoch(self:BarlowTwins):

    if self.epoch % 100 == 0:
        pass
        
        #self.learn.save(f'bt_{self.epoch}')
        #save the state_dict of the model
        #probably a unique folder for each experiment?
        #


# model = train_bt(model=model,
#                 dls=dls,
#                 bt_aug_pipelines=bt_aug_pipelines,
#                 lmb=lmb,
#                 n_in=config.n_in,
#                 model_type=config.model_type,
#                 wd=config.wd,
#                 epochs=config.epochs,
#                 freeze_epochs=config.freeze_epochs,
#                 weight_type=config.weight_type,
#                 device=device,

#                 ) 


KeyboardInterrupt: 

In [ ]:
# import os
# from datetime import datetime

# # Example structure: /content/drive/My Drive/Experiments/cifar10/resnet18/2023-01-01_12-00-00/
# base_dir = '/content/drive/My Drive/Experiments'
# experiment_dir = os.path.join(base_dir, config.dataset, config.arch, datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

# epoch=100
# checkpoint_path = os.path.join(experiment_dir, f'model_epoch_{epoch}.pt')
# checkpoint_path

'/content/drive/My Drive/Experiments/cifar10/resnet18/2024-02-10_19-05-29/model_epoch_100.pt'

First, get some MNIST data, define a model and augmentation pipeline:

In [ ]:
#| hide
#Get some MNIST data and plonk it into a dls
# bs=32
# path = untar_data(URLs.MNIST)
# items = get_image_files(path/'training') #i.e. NOT testing!!!
# items=items.shuffle()
# items = items[0:bs*2]
# split = RandomSplitter(valid_pct=0.0)
# tds = Datasets(items, [PILImageBW.create, [parent_label, Categorize()]], splits=split(items))
# dls = tds.dataloaders(bs=bs,num_workers=0, after_item=[ToTensor(), IntToFloatTensor()], device=default_device().type)

# #Define the model
# size=28
# ps=10
# hs=10
# n_in=1
# fastai_encoder = create_fastai_encoder(xresnet18(),pretrained=False,n_in=n_in)
# #model = create_barlow_twins_model(fastai_encoder, hidden_size=hs,projection_size=ps)# projection_size=1024)
# model = create_p3barlow_twins_model(fastai_encoder, hidden_size=hs,projection_size=ps)# projection_size=1024)


# #Aug pipeline
# aug_pipelines_1 = get_barlow_twins_aug_pipelines(size=28,blur_r=(0.5,2.5),
#                     rotate=True,jitter=False,noise=False,bw=False,blur=True,solar=False, #Whether to use aug or not
#                     resize_scale=(0.5, 1.0),resize_ratio=(3/4, 4/3),noise_std=0.05, rotate_deg=25,blur_s=11,sol_t=0.025,sol_a=0.025, #hps of augs
#                     flip_p=0.1, rotate_p=0.5,noise_p=0.5, jitter_p=0.3, bw_p=0.3, blur_p=0.5,sol_p=0.2, #prob of performing aug
#                     same_on_batch=False,stats=mnist_stats)

# aug_pipelines_2 = get_barlow_twins_aug_pipelines(size=28,blur_r=(0.5,2.5),
#                     rotate=True,jitter=False,noise=True,bw=False,blur=False,solar=True, #Whether to use aug or not
#                     resize_scale=(0.5, 1.0),resize_ratio=(3/4, 4/3),noise_std=0.05, rotate_deg=25,blur_s=11,sol_t=0.025,sol_a=0.025, #hps of augs
#                     flip_p=0.1, rotate_p=0.5,noise_p=0.5, jitter_p=0.3, bw_p=0.3, blur_p=1.0,sol_p=0.2, #prob of performing aug
#                     same_on_batch=False,stats=mnist_stats)

#"union" if 1 and 2
# aug_pipelines_12 = get_barlow_twins_aug_pipelines(size=28,blur_r=(0.5,2.5),
#                     rotate=True,jitter=False,noise=True,bw=False,blur=True,solar=True, #Whether to use aug or not
#                     resize_scale=(0.5, 1.0),resize_ratio=(3/4, 4/3),noise_std=0.05, rotate_deg=25,blur_s=11,sol_t=0.025,sol_a=0.025, #hps of augs
#                     flip_p=0.1, rotate_p=0.5,noise_p=0.5, jitter_p=0.3, bw_p=0.3, blur_p=0.5,sol_p=0.2, #prob of performing aug
#                     same_on_batch=False,stats=mnist_stats)

#aug_pipelines = [aug_pipelines_1,aug_pipelines_1]
# tem = BarlowTwins(aug_pipelines,n_in=n_in,print_augs=True)
# learn = Learner(dls,model, cbs=[tem])
# #learn.fit(2)

In [ ]:
#| hide
#show_bt_batch(dls=dls,n_in=1,aug=aug_pipelines,n=2,print_augs=True)

In [ ]:
#| hide


#train_bt = TrainBT(model=model,hp_dict={'dls':dls,'aug_pipelines':aug_pipelines,'wd':1.5*1e-6,n_in':n_in,'lmb':1/ps,'model_type':'barlow_twins'})
#model=train_bt.bt_learning(epochs=1)

For completeness, here is an example on CIFAR10

In [ ]:
# #| hide
# n_in=3
# size=32
# bs=100
# path = untar_data(URLs.CIFAR)
# items = get_image_files(path / "train")
# items=items.shuffle()
# items = items[0:200]
# split = RandomSplitter(valid_pct=0.0) #randomly split training set into training and validation
# #tds = Datasets(items,splits=split(items)) #Do we want this?
# tds = Datasets(items, [[PILImage.create,Resize(size)], [parent_label, Categorize()]], splits=split(items)) #Or do we want this?

# dls = tds.dataloaders(bs=bs,num_workers=0, after_item=[ToTensor(), IntToFloatTensor()], device=default_device().type)

# fastai_encoder = create_fastai_encoder(xresnet18(),pretrained=False,n_in=n_in)
# model = create_barlow_twins_model(fastai_encoder, hidden_size=10,projection_size=10)# projection_size=1024)

# aug_pipelines_1 = get_barlow_twins_aug_pipelines(size=size,
#                                                  bw=True, rotate=True,noise=True, jitter=True, blur=True,solar=True,
#                                                  resize_scale=(0.4, 1.0),rotate_deg=45,noise_std=0.0125, jitter_s=1.0, blur_s=math.ceil(size/10)+1,
#                                                  bw_p=0.2, flip_p=0.5,rotate_p=0.25,noise_p=0.5, jitter_p=0.5, blur_p=0.5,sol_p=0.0,
#                                                  stats=cifar_stats,same_on_batch=False, xtra_tfms=[]
#                                                  )

# aug_pipelines_2 = get_barlow_twins_aug_pipelines(size=size,
#                                                  bw=True, rotate=True,noise=True, jitter=True, blur=True,solar=True,
#                                                  resize_scale=(0.4, 1.0),rotate_deg=45,noise_std=0.0125, jitter_s=1.0, blur_s=math.ceil(size/10)+1,sol_t=0.01,sol_a=0.01,
#                                                  bw_p=0.2, flip_p=0.5,rotate_p=0.25,noise_p=0.5, jitter_p=0.5, blur_p=0.1,sol_p=0.2,
#                                                  stats=cifar_stats,same_on_batch=False, xtra_tfms=[]
#                                                  )

# aug_pipelines = [aug_pipelines_1,aug_pipelines_2]

# aug_pipelines = [aug_pipelines_1,aug_pipelines_2]

# aug_pipelines = [aug_pipelines_1,aug_pipelines_2]
# learn = Learner(dls,model, cbs=[BarlowTwins(aug_pipelines,n_in=n_in,print_augs=True)])
# #learn.fit(1)

Here are the augmentations used:

In [ ]:
#| hide
#show_bt_batch(dls=dls,n_in=n_in,aug=aug_pipelines,n=20,print_augs=True)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()